# Introduction 🎢

This notebook will be mainly used for the capstone project.

# Coding Fun 🏖️💻

_Let's import the usual suspects_ 🧙‍♀️🧛🏻‍♂️🧟‍♂️ 🤓:

In [1]:
import pandas as pd
import numpy as np

_... and some new friends_  🦕🐢🐙🐿️:

In [68]:
import geocoder
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if not installed on your system
from geopy.geocoders import Nominatim # convert an address into latitude and l ongitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas d ataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if not installed on your system
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


---

# 3️⃣

# Segmenting and Clustering Neighborhoods in Toronto

In [17]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)[0]
df.shape

(287, 3)

In [18]:
df.head(3)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


Get rid off 🔫 that stupid `Not assigned` and fill it with `NaN`:

In [19]:
df = df.replace("Not assigned", np.nan)

##### a) Remove data without Bouroughs
>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [20]:
df = df[~df["Borough"].isna()]
df.shape

(210, 3)

#### b) Clean Neigbourhood Column
>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

_This stupid task took me almost 1 day!_ 😤 _to figure out how - trying different approaches!!! Nothing really worked out._ 😠 💣🔪🔫 _Now, I hate Pandas!_ 🐼

In [21]:
# using pandas built-in aggregate function did the trick
df = df.groupby('Postcode').agg(Bourough=("Borough","first"), 
                                 Neighbourhood=('Neighbourhood', list)).reset_index()
df.shape

(103, 3)

In [30]:
df.head()

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


#### c) Replace not assigned Neigbourhoods with Bouroughs
> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [24]:
#replace na in Neigbourhood with value in Bourough
df['Neighbourhood'].fillna(value=df['Bourough'],inplace=True)

In [31]:
df.head(5)

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


---

In [32]:
df.shape

(103, 3)

In [34]:
# Save DataFrame as file 
df.to_pickle("df_cleaned.pkl")

## Adding geo coordinates for Neighbourhoods

_First of all, I read the saved DataFrame:_ 📖👓

In [61]:
df = pd.read_pickle("df_cleaned.pkl")

_Okay. I admit: I'm lazy._ 🛌 _So I use the prepared table with the coordinates._

In [63]:
# I downloaded table from "https://cocl.us/Geospatial_data" as csv
coord = pd.read_csv("data/Geospatial_Coordinates.csv")
coord.shape

(103, 3)

In [64]:
# rename column "Postal Code" -> "Postcode" for merging
coord.rename(columns={"Postal Code": "Postcode"}, inplace=True)

In [65]:
coord.head(2)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


Merge both Dataframes

In [66]:
df = pd.merge(df, coord, how='left', on='Postcode')

In [67]:
df.head(5)

,Postcode,Bourough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,43.773136,-79.239476


--- 

## Explore and cluster the neighborhoods in Toronto

>Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.   Just make sure:   
>- to add enough Markdown cells to explain what you decided to do and to report any observations you make.   
>- to generate maps to visualize your neighborhoods and how they cluster together.

## Download Foursquare Data for Toronto

In order to use Foursquares REST API you need to open a (free) account. After this you get an  Read my Login Information for Foursquare



In [86]:
# Read my Login Information for Foursquare
file = "data/my_secret_foursquare_login_data.json"
with open (file) as f:
    data = f.read()
login_data = json.loads(data)

In [87]:
CLIENT_ID = login_data["CLIENT_ID"]
CLIENT_SECRET = login_data["CLIENT_SECRET"]

In [96]:
# extracting the average coordinates for the toronto map
long = df.Longitude_x.mean()
lat = df.Latitude_x.mean()

-79.39715291165048

_Where is Toronto?_ (Of course, in Canada 🇨🇦, but where exatly? Sorry 🙇🏻‍♂️ Canadians for my ignorance)

In [118]:
 # define a world map centered around Toronto with a low zoom level
toronto_map = folium.Map(location=[lat, long], zoom_start=2)
folium.features.CircleMarker(
            [lat, long],
            radius=10, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1).add_to(toronto_map)
# display world map
toronto_map

_Oh - I didn't know it is so near to the US boarder. Maybe this could be interisting for further investigations ..._ 🤔

# Playing around

In [ ]:
neighborhood_latitude = df[df["Poscode"]=="M1B"].
neighborhood_longitude
    
    
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

In [119]:
df.head()

,Postcode,Bourough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,43.773136,-79.239476


---
# 🔚 🙏🏼 🙋🏻‍♂️